# ApuliaTravelKG

In [2]:
import pandas as pd
import json

from rdflib import Graph
from math import radians, sin, cos, sqrt, atan2

In [3]:
INPUT_FILE = "TravelPugliaKG.ttl"

g = Graph()
g.parse(INPUT_FILE, format="ttl")

<Graph identifier=N761191008ea541778595044b842c8a08 (<class 'rdflib.graph.Graph'>)>

**CQ 1**: How far are POIs from a given location (expressed as lat and long)?

In [ ]:
# impl 1 - using GeoSPARQL functions (if supported by the triplestore)
cq1_impl1 = """
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
PREFIX uom: <http://www.opengis.net/def/uom/OGC/1.0/>
PREFIX wgs: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX poi: <https://w3id.org/italia/onto/POI/>
PREFIX clv: <https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?poi ?lat ?long ?distance
WHERE {
  ?poi a poi:PointOfInterest ;
         clv:hasGeometry ?geom .
  ?geom wgs:lat ?lat ;
        wgs:long ?long .

  BIND(CONCAT("POINT("?long, " ", ?lat, ")")^^geo:wktLiteral AS ?poiGeom)

  BIND("POINT(13.4050 52.5200)"^^geo:wktLiteral AS ?refGeom)

  BIND(geof:distance(?poiGeom, ?refGeom, uom:metre) AS ?distance)
}
"""

# impl 2 - using Haversine (distance) formula implemented in Python (if GeoSPARQL functions are not supported)
cq1_impl2 = """
PREFIX clv: <https://w3id.org/italia/onto/CLV/>
PREFIX poi: <https://w3id.org/italia/onto/POI/>

SELECT DISTINCT ?poi ?lat ?long
WHERE {
  ?poi a poi:PointOfInterest .
  ?poi clv:hasGeometry ?geom .
  ?geom a clv:Geometry .
  ?geom clv:lat ?lat ;
        clv:long ?long .
}
LIMIT 10
"""

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # metres
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

results = g.query(cq1_impl2)
df = pd.DataFrame(results.bindings)
df

,lat,long,poi
0,"41,127743","16,86676",https://w3id.org/italia/onto/POI/PointOfIntere...
1,"41,321297","16,288715",https://w3id.org/italia/onto/POI/PointOfIntere...
2,"40,144684","18,492456",https://w3id.org/italia/onto/POI/PointOfIntere...
3,"41,630486","15,922593",https://w3id.org/italia/onto/POI/PointOfIntere...
4,"40,27353","18,043858",https://w3id.org/italia/onto/POI/PointOfIntere...
5,"40,498665","17,642132",https://w3id.org/italia/onto/POI/PointOfIntere...
6,"41,5093","15,321773",https://w3id.org/italia/onto/POI/PointOfIntere...
7,"40,640564","17,937218",https://w3id.org/italia/onto/POI/PointOfIntere...
8,"40,473267","17,233543",https://w3id.org/italia/onto/POI/PointOfIntere...
9,"41,281887","16,415602",https://w3id.org/italia/onto/POI/PointOfIntere...


**CQ 2**: Which is the score of a given POI for a given aspect?

In [66]:
cq2 = """
PREFIX sm: <https://w3id.org/italia/onto/SM/>
PREFIX poi: <https://w3id.org/italia/onto/POI/>
 
SELECT DISTINCT ?poi ?rating
WHERE {
    ?poi a poi:PointOfInterest .
    ?poi sm:hasReview ?review .
    ?review a sm:Review .
    ?review sm:reviewAspect "Arte e Cultura" .
    ?review sm:hasRating ?rating .
}
LIMIT 10
"""

results = g.query(cq2)
df = pd.DataFrame(results.bindings)
df

,poi,rating
0,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
1,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
2,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
3,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
4,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
5,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
6,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
7,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
8,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5
9,https://w3id.org/italia/onto/POI/PointOfIntere...,https://w3id.org/italia/onto/SM/Rating/5


**CQ 3**: Which POIs are open on a given day of the week?

In [67]:
DAY = "Monday"

cq3 = """
PREFIX clv: <https://w3id.org/italia/onto/CLV/>
PREFIX ac: <https://w3id.org/italia/onto/AccessCondition/>
PREFIX ti: <https://w3id.org/italia/onto/TI/>
PREFIX poi: <https://w3id.org/italia/onto/POI/>

SELECT ?poi
WHERE {
    ?poi a poi:PointOfInterest .
    ?poi ac:hasAccessCondition ?accessCondition .
    ?accessCondition a clv:OpeningHoursSpecification .
    ?accessCondition ti:hasDayOfWeek ti:Monday .
}
LIMIT 10
"""

results = g.query(cq3)
df = pd.DataFrame(results.bindings)
df

,poi
0,https://w3id.org/italia/onto/POI/PointOfIntere...
1,https://w3id.org/italia/onto/POI/PointOfIntere...
2,https://w3id.org/italia/onto/POI/PointOfIntere...
3,https://w3id.org/italia/onto/POI/PointOfIntere...
4,https://w3id.org/italia/onto/POI/PointOfIntere...
5,https://w3id.org/italia/onto/POI/PointOfIntere...
6,https://w3id.org/italia/onto/POI/PointOfIntere...
7,https://w3id.org/italia/onto/POI/PointOfIntere...
8,https://w3id.org/italia/onto/POI/PointOfIntere...
9,https://w3id.org/italia/onto/POI/PointOfIntere...


**CQ 4**: Which POIs are museums?

In [5]:
cq4 = """
PREFIX td: <https://example.org/td/>

SELECT ?td
WHERE {
    ?td a td:Museum .
}
LIMIT 10
"""

results = g.query(cq4)
df = pd.DataFrame(results.bindings)
df

,td
0,https://w3id.org/italia/onto/POI/PointOfIntere...
1,https://w3id.org/italia/onto/POI/PointOfIntere...
2,https://w3id.org/italia/onto/POI/PointOfIntere...
3,https://w3id.org/italia/onto/POI/PointOfIntere...
4,https://w3id.org/italia/onto/POI/PointOfIntere...
5,https://w3id.org/italia/onto/POI/PointOfIntere...
6,https://w3id.org/italia/onto/POI/PointOfIntere...
7,https://w3id.org/italia/onto/POI/PointOfIntere...
8,https://w3id.org/italia/onto/POI/PointOfIntere...
9,https://w3id.org/italia/onto/POI/PointOfIntere...
